In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tabulate as tb
import matplotlib.pyplot as plt
import seaborn as sns


CSV_PATH = "./../data/GOOGL.csv"
LOOKBACK = 30               # number of past days used as input
INITIAL_TRAIN_DAYS = 5000   # number of days to use initially for training (or use fraction)
BATCH_SIZE = 32
INITIAL_EPOCHS = 30
UPDATE_EPOCHS = 2
LEARNING_RATE = 1e-3
FINE_TUNE_LR = 1e-4
MODEL_SAVE_PATH = "lstm_multi_step.h5"
SCALER_SAVE_PATH = "scaler.save"

FEATURES = [
   'Open', 'High', 'Low', 'Close', 'Volume',
   'rsi', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
   'BBL_20_2.0_2.0','BBM_20_2.0_2.0','BBU_20_2.0_2.0','BBB_20_2.0_2.0','BBP_20_2.0_2.0',
   'ema_10','ema_20','ema_50','atr','stoch_k','stoch_d'
]
TARGET = 'Close'


In [ ]:
df = pd.read_csv(CSV_PATH, parse_dates=['Date'], index_col='Date')
df = df.sort_index()

X, y = [], []
    

print(tb.tabulate(df.head(), headers='keys', tablefmt='psql'))

+---------------------+---------+---------+---------+---------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+----------+-----------+-----------+-------------+
| Date                |   Close |    High |     Low |    Open |      Volume |     rsi |   MACD_12_26_9 |   MACDh_12_26_9 |   MACDs_12_26_9 |   BBL_20_2.0_2.0 |   BBM_20_2.0_2.0 |   BBU_20_2.0_2.0 |   BBB_20_2.0_2.0 |   BBP_20_2.0_2.0 |   ema_10 |   ema_20 |   ema_50 |      atr |   stoch_k |   stoch_d |     returns |
|---------------------+---------+---------+---------+---------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+----------+-----------+-----------+-------------|
| 2004-10-28 00:00:00 | 4.8042  | 4.83129 | 4.